In [ ]:
import pandas as pd
import numpy as np
import csv
import re

In [ ]:
df=pd.read_csv('Parking_Bays_20240222.csv',header=0)

In [ ]:
# 1) Print all headers
col_ind = [(i, col) for i, col in enumerate(df.columns.to_list())]
print(col_ind)

In [ ]:
# 2) Drop all columns unnecessary columns: ['Times Of Operations'], ['Maximum Stay'], ['Cashless Identifier'], ['Nearest Machine'], [9:]
df1 = df.iloc[:,[0,1,4,7,8,21]]

In [ ]:
# 3) Print all unique values in 'Restriction Type' 
header_names_filt = df1['Restriction Type'].unique()
print(header_names_filt)

In [ ]:
# 4) Drop all rows corresponding to unpaid parking
df2 = df1[df['Restriction Type'].str.contains('paid')]
print('There are',df2.shape[0],' paid, parking spaces in Camden')
print(df2['Tariff'].unique())

In [ ]:
# 5) Split ['Tariff'] into ['Petrol Tariff'] and ['Diesel Tariff'], splitting at the '/'
df2[['Non-diesel Tariff', 'Diesel Tariff']] = df2['Tariff'].str.split('/', expand=True)
col_ind1 = [(i, col) for i, col in enumerate(df2.columns.to_list())]
print(col_ind1)

In [ ]:
# 6) Drop ['Tariff']
df3 = df2.iloc[:,[0, 1, 3, 4, 5, 6, 7]]
print(df3)

In [ ]:
# 7) The value '£8.39 p.h. (buses and coaches)' exists, and has left blanks in ['Diesel Tariff'], copy the value over to ['Petrol Tariff']
df3['Diesel Tariff'].fillna('£8.39 (diesel)', inplace=True)

# 8) Remove excess text and clear blank-spaces from ['Petrol Tariff'] and ['Diesel Tariff'] so just numbers remain
def keep_numeric(text):
    cleaned_text = ''
    decimal_point_seen = False
    for char in text:
        if char.isdigit():
            cleaned_text += char
        elif char == '.' and not decimal_point_seen:
            cleaned_text += char
            decimal_point_seen = True
    return cleaned_text
df3[['Non-diesel Tariff', 'Diesel Tariff']] = df3[['Non-diesel Tariff', 'Diesel Tariff']].apply(lambda x: x.str.strip())

# 9) Convert ['Petrol Tariff'] and ['Diesel Tariff'] data types to float
df3[['Non-diesel Tariff', 'Diesel Tariff']] = df3[['Non-diesel Tariff', 'Diesel Tariff']].apply(lambda x: x.apply(keep_numeric))
                                                      
print(df3)

In [ ]:
# 10) Check for, and delete blanks 
blank_check = df3.isna().any()
columns_with_missing_values = blank_check[blank_check].index.tolist()
for column in columns_with_missing_values:
    df3[column].fillna(0, inplace=True)

for column in columns_with_missing_values:
    print(df3[df3[column].isna()])

df4 = df3[(df3[columns_with_missing_values] != 0).any(axis=1)]
print(df4)

In [ ]:
# 11) Re-name and Re-order columns
df4 = df4.rename(columns={'Diesel Tariff': 'Diesel Tariff (£)', 'Non-diesel Tariff': 'Non-diesel Tariff (£)'})
df5 = df4[['Restriction Type','Parking Spaces','Road Name','Postcode','Non-diesel Tariff (£)','Diesel Tariff (£)','Unique Identifier']]
df5 = df5.sort_values(by='Road Name')
df5 = df5.reset_index(drop=True)
print(df5)

In [ ]:
# 12) Save as new, cleaned CSV file
df5.to_csv('CLEANED_Parking_Bays_20240222.csv',index=False)